In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d01a9912-704e-34e2-d24b-4644525bfbd2)


#install pacakges

In [2]:
%%capture
!python3 -m pip install -U pip
!python3 -m pip install -U setuptools wheel
!python3 -m pip install -U "mxnet_cu110<2.0.0"
!python3 -m pip install autogluon

In [3]:
%%capture
!pip install -U transformers

# download data

In [4]:
import os
if os.path.isdir('/content/Profiling-Hate-Speech-Spreaders-on-Twitter') is False:
    !git clone https://github.com/talhaanwarch/Profiling-Hate-Speech-Spreaders-on-Twitter.git

Cloning into 'Profiling-Hate-Speech-Spreaders-on-Twitter'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 31 (delta 6), reused 6 (delta 1), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [5]:
with open('pass.txt','r') as f:
  pas=f.read()


In [6]:
import zipfile
if os.path.isdir('train') is False:
  train_path='/content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/train.zip'
  with zipfile.ZipFile(train_path,"r") as zip_ref:
    zip_ref.extractall(".",pwd = bytes(pas,'utf-8'))
if os.path.isdir('test') is False:
  test_path='/content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/test.zip'
  with zipfile.ZipFile(test_path,"r") as zip_ref:
    zip_ref.extractall(".",pwd = bytes(pas,'utf-8'))

# prepare data

## train data

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df=pd.read_csv('train/en/truth.txt',sep=':::',header=None,engine='python')
df.columns=['id','label']

In [8]:
df.label.value_counts()

1    100
0    100
Name: label, dtype: int64

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
def preprocessing(text):
    text=text.str.replace('\d+', '')
    text=text.str.replace('RT','')
    text=text.str.replace('#USER#','')
    text=text.str.replace('_','')
    text=text.str.replace('#URL#','')
    text= text.str.lower()
    text = text.str.replace('[^\w\s]','')
    #text = text.apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
    #text = text.apply(lambda x: [item for item in x if item not in stop])
    #text = text.apply(lambda x : " ".join(x))
    return text

In [11]:
df.id='train/en/'+df.id.astype(str)+'.xml'
df.head()

,id,label
0,train/en/639b8e5e6a527d494c85d8f5704b1a01.xml,0
1,train/en/f2b1fc84c500c38a93522efbd422b559.xml,0
2,train/en/10b2d013382e1fb3c9414ea28329f258.xml,0
3,train/en/26644d1348fc1122e8c5ef45d6bc84fa.xml,0
4,train/en/4d4c5dcbfe38d0d33a0d1b1419952ca8.xml,0


In [12]:
len(df)

200

In [13]:
import xml.etree.ElementTree as ET
def reader(df,ground=True):
  data=[]
  for x in df.iterrows():
      
      tree = ET.parse(x[1].id)
      root = tree.getroot()
      text=[x.text for x in root[0]]
      if ground:
        label=[x[1].label]*len(text)
        data.append(pd.DataFrame(zip(text,label),columns=['text','label']))
      else:
        data.append(pd.DataFrame(text,columns=['text']))
  return data

In [14]:
data=reader(df)

In [15]:
data[0].head()

,text,label
0,RT #USER#: Funny how “15 days to slow the spre...,0
1,RT #USER#: Why did Minneapolis just give Georg...,0
2,"RT #USER#: To be fair, he has done a lot of un...",0
3,RT #USER#: President Trump got us the #HASHTAG...,0
4,RT #USER#: Is the case against former officer ...,0


In [16]:

datax=data.copy()
for d in range(len(data)):
  datax[d].text=preprocessing(data[d].text)


In [17]:
datax[0].head()

,text,label
0,funny how days to slow the spread turned in...,0
1,why did minneapolis just give george floyds ...,0
2,to be fair he has done a lot of undercover w...,0
3,president trump got us the hashtag vaccine p...,0
4,is the case against former officer derek cha...,0


In [18]:
len(datax[0])

200

In [19]:
doc_text,doc_label,=[],[]
for i in range(len(datax)):
  doc_text.append(list(datax[i].text))
  doc_label.append(int(datax[i].mean()))

In [20]:
len(doc_text),len(doc_label)

(200, 200)

## test data

In [21]:
from glob import glob
test_path=pd.DataFrame(glob('test/en/*.xml'),columns=['id'])
test_path.head()

,id
0,test/en/f2973063f16c0698a6de86c55b5f9ad6.xml
1,test/en/be71e41921653e523aceb555727b61c7.xml
2,test/en/d0235c3e74f79298fc2b6d2427c2060b.xml
3,test/en/314d4bf068314367bc83dc5ae3757e13.xml
4,test/en/fdd27881da241cff7115c1e89caba6dd.xml


In [22]:
test_data=reader(test_path,ground=False)

In [23]:

test_datax=test_data.copy()
for d in range(len(test_datax)):
  test_datax[d].text=preprocessing(test_datax[d].text)


In [24]:
test_doc=[]
for i in range(len(test_datax)):
  test_doc.append(list(test_datax[i].text))

In [25]:
len(test_doc)

100

# generate embeddings

In [26]:
from transformers import AutoTokenizer, AutoModel
from torch.nn.utils.rnn import pad_sequence
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased",)
model = AutoModel.from_pretrained("bert-base-uncased",output_hidden_states=True)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
device='cuda'
model=model.to(device)

In [28]:
#get data from last hidden layer
def create_embeddings(doc_text,max_length=20,cat=False):
  doc_embeddings=[]
  for text in doc_text:
    inputs=tokenizer(text,max_length=max_length,padding=True,truncation=True)
    text_seq = torch.tensor(inputs['input_ids']).to(device)
    text_mask = torch.tensor(inputs['attention_mask']).to(device)
    model.eval()
    with torch.no_grad():
      outputs= model(text_seq,attention_mask=text_mask)
      hidden_states=outputs.hidden_states
      if cat:
        emb=torch.mean(torch.cat([hidden_states[i] for i in [-1,-2,-3,-4]], dim=-1),1)
      else:
        emb=torch.mean(hidden_states[11], 1)    
      doc_embeddings.append(emb)
  x=torch.mean(torch.stack(doc_embeddings),1)
  x=np.array(x.cpu())
  return x

## last layer

In [29]:
%%time
train_last=create_embeddings(doc_text)
test_last=create_embeddings(test_doc)


CPU times: user 1min 2s, sys: 291 ms, total: 1min 2s
Wall time: 1min 1s


## last 4 layers

In [30]:
%%time
train_last4=create_embeddings(doc_text,cat=True)
test_last4=create_embeddings(test_doc,cat=True)


CPU times: user 1min 5s, sys: 142 ms, total: 1min 5s
Wall time: 1min 4s


# AutoGluon

In [31]:
def agdataframe(train,label,test):
  train_df=pd.concat([pd.DataFrame(train),pd.DataFrame(label)],axis=1)
  train_df.columns=['col_{}'.format(i) for i in range(train.shape[1])]+['Label']
  train_df = train_df.sample(frac=1).reset_index(drop=True)
  test_df=pd.DataFrame(test)
  test_df.columns=['col_{}'.format(i) for i in range(test.shape[1])]
  return train_df, test_df
  

In [32]:
train_last_df,test_last_df=agdataframe(train_last,doc_label,test_last)
train_last4_df,test_last4_df=agdataframe(train_last4,doc_label,test_last4)

print(train_last_df.shape,test_last_df.shape,train_last4_df.shape,test_last4_df.shape)

(200, 769) (100, 768) (200, 3073) (100, 3072)


In [33]:
from autogluon.tabular import  TabularPredictor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,accuracy_score

def loop(data,test):
  acc=[]
  test_pred=[]
  if os.path.isdir('AutogluonModels'):
    !rm -rf AutogluonModels
  skf = StratifiedKFold(n_splits=5,shuffle=True)
  for train_index, val_index in skf.split(data.iloc[:,0:-1], data.iloc[:,-1]):
    train=data.iloc[train_index,:]
    val=data.iloc[val_index,:]
    clf=TabularPredictor(label='Label',verbosity=0).fit(train, presets='best_quality')
    res=clf.leaderboard(val,silent=True)
    res['score_test']=res['score_test'].round(3)*100
    pred = clf.predict(val,model=res.model[0])
    accuracy=accuracy_score(val.Label,pred)
    acc.append(accuracy)
    print('best model ',res.model[0],' accuracy is ',accuracy)
    test_pred.append(clf.predict(test,model=res.model[0]))
  print('average accuracy is',np.mean(acc))
  return test_pred,np.mean(acc)

In [34]:
%%time
test_pred_last,test_acc_last=loop(train_last_df,test_last_df)

best model  XGBoost_BAG_L1  accuracy is  0.75
best model  XGBoost_BAG_L1  accuracy is  0.8
best model  LightGBM_BAG_L1  accuracy is  0.7
best model  LightGBM_BAG_L1  accuracy is  0.75
best model  LightGBMXT_BAG_L1  accuracy is  0.75
average accuracy is 0.75
CPU times: user 23min 40s, sys: 14.9 s, total: 23min 54s
Wall time: 14min 9s


In [38]:
print(len(np.mean(test_pred_last,axis=0)),len(test_last_df)) #100

100 100


In [39]:
%%time
test_pred_last4,test_acc_last4=loop(train_last4_df,test_last4_df)

best model  ExtraTreesEntr_BAG_L1  accuracy is  0.7
best model  ExtraTreesGini_BAG_L1  accuracy is  0.675
best model  NeuralNetMXNet_BAG_L1  accuracy is  0.675
best model  NeuralNetMXNet_BAG_L1  accuracy is  0.825
best model  LightGBM_BAG_L1  accuracy is  0.75
average accuracy is 0.725
CPU times: user 1h 22min 41s, sys: 28.2 s, total: 1h 23min 10s
Wall time: 48min 11s


In [40]:
def make_final(test_pred):
  ids=test_path.id.apply(lambda x:x.split('/')[-1].split('.')[0])
  pred=np.mean(test_pred,axis=0)
  lang=['en']*len(test_path)
  final=pd.DataFrame(zip(ids,lang,pred),columns=['author-id','lang','type'])
  return final

In [41]:
test_last=make_final(test_pred_last)
test_last4=make_final(test_pred_last4)

In [42]:
test_last.shape,test_last4.shape

((100, 3), (100, 3))

In [43]:
test_last.to_csv('bertbase1_{}.csv'.format(test_acc_last),index=False)
test_last4.to_csv('bertbase2_{}.csv'.format(test_acc_last4),index=False)

In [ ]:
if os.path.isdir('output'):
  !rm -rf output
  !mkdir output
  !rm talha.zip
else:
  !mkdir output

In [ ]:
import pandas as pd
df=final.copy()
import pandas as pd
for row in df.index:
    xml = []
    xml.append('<author id="{}"'.format(df.loc[row,'author-id']))
    xml.append('lang="{}"'.format(df.loc[row,'lang']))
    xml.append('type="{}"'.format(df.loc[row,'type']))
    xml.append('/>')
    x='\n'.join(xml)
    myfile = open("output/{}.xml".format(df.loc[row,'author-id']), "w")
    myfile.write(x)
    myfile.close()
    

In [ ]:
%%capture
!zip -r talha.zip output